In [1]:
# Cell 1: Install dependencies
!pip install openai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.7 MB/s 

In [2]:
# Cell 2: Load the JSON data
import json
import os

file_path = "/content/rag_dataset_final.json"

if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    print(f"✅ Loaded {len(json_data)} items from {file_path}")
else:
    print(f"❌ File not found at {file_path}. Please check the path.")
    json_data = []

✅ Loaded 8347 items from /content/rag_dataset_final.json


In [ ]:
# Cell 3: Setup Fireworks Client & Embedding Function
from openai import OpenAI

# Initialize Client
# ⚠️ Note: Using the key provided in your script
client = OpenAI(
    api_key="FIREWORKS_API_KEY",
    base_url="https://api.fireworks.ai/inference/v1"
)

EMBEDDING_MODEL = "accounts/fireworks/models/qwen3-embedding-8b"

def get_fireworks_embeddings(text_list):
    """
    Generates embeddings using Fireworks API.
    Accepts a list of strings and returns a list of vectors.
    """
    # API usually handles newlines better if replaced, but optional depending on model
    clean_texts = [t.replace("\n", " ") for t in text_list]

    response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=clean_texts
    )

    # Extract embeddings ensuring order is preserved
    return [data.embedding for data in response.data]

print("✅ Fireworks Embedding Client Ready.")

✅ Fireworks Embedding Client Ready.


In [4]:
import pydantic
print(pydantic.__version__)

2.12.3


In [5]:
import sys, pydantic
print("Python path:", sys.executable)
print("Pydantic version:", pydantic.__version__)

Python path: /usr/bin/python3
Pydantic version: 2.12.3


In [6]:
# Cell 4: Create Vector Store
import chromadb
import time

# 1. Setup Database Client
db_client = chromadb.PersistentClient(path="./my_arabic_db")

# 2. Reset Collection (Delete old one to ensure schema matches new model)
try:
    db_client.delete_collection(name="grammar_collection")
    print("🗑️ Old collection deleted.")
except:
    pass

collection = db_client.get_or_create_collection(name="grammar_collection")

# 3. Prepare Data
documents = []
ids = []
metadatas = []

if json_data:
    print("Preparing data lists...")
    for i, item in enumerate(json_data):
        q_text = item.get('question_without_diacritics', item.get('question_with_diacritics', ''))
        a_text = item.get('answer_without_diacritics', item.get('answer_with_diacritics', ''))

        text_content = f"Question: {q_text}\nAnswer: {a_text}"

        documents.append(text_content)
        ids.append(str(item.get('id', i)))
        metadatas.append({"source": "rag_dataset"})

    # 4. Generate Embeddings & Upsert in Batches
    # We use a smaller batch size (e.g., 50) for API stability
    BATCH_SIZE = 100
    total_docs = len(documents)

    print(f"⚡ Processing {total_docs} documents via Fireworks API...")

    for i in range(0, total_docs, BATCH_SIZE):
        end = min(i + BATCH_SIZE, total_docs)

        batch_docs = documents[i:end]
        batch_ids = ids[i:end]
        batch_meta = metadatas[i:end]

        print(f"   Processing batch {i} to {end}...")

        try:
            # Get Embeddings from API
            batch_embeddings = get_fireworks_embeddings(batch_docs)

            # Save to ChromaDB
            collection.upsert(
                documents=batch_docs,
                embeddings=batch_embeddings,
                ids=batch_ids,
                metadatas=batch_meta
            )
        except Exception as e:
            print(f"❌ Error on batch {i}: {e}")
            # Optional: Sleep briefly if hitting rate limits
            time.sleep(10)

    print("✅ Vector store created successfully!")
else:
    print("⚠️ No data found to process.")

Preparing data lists...
⚡ Processing 8347 documents via Fireworks API...
   Processing batch 0 to 100...
   Processing batch 100 to 200...
   Processing batch 200 to 300...
   Processing batch 300 to 400...
   Processing batch 400 to 500...
   Processing batch 500 to 600...
   Processing batch 600 to 700...
   Processing batch 700 to 800...
   Processing batch 800 to 900...
   Processing batch 900 to 1000...
   Processing batch 1000 to 1100...
   Processing batch 1100 to 1200...
   Processing batch 1200 to 1300...
   Processing batch 1300 to 1400...
   Processing batch 1400 to 1500...
   Processing batch 1500 to 1600...
   Processing batch 1600 to 1700...
   Processing batch 1700 to 1800...
   Processing batch 1800 to 1900...
   Processing batch 1900 to 2000...
   Processing batch 2000 to 2100...
   Processing batch 2100 to 2200...
   Processing batch 2200 to 2300...
   Processing batch 2300 to 2400...
   Processing batch 2400 to 2500...
   Processing batch 2500 to 2600...
   Processin

In [7]:
'''# Cell 5: Strict Chat System (Anti-Hallucination)
import chromadb

# 1. Connect to Database
db_client = chromadb.PersistentClient(path="./my_arabic_db")
collection = db_client.get_collection(name="grammar_collection")

# 2. Retrieval Function with Similarity Check (same as before)
def retrieve_strict_context(query, threshold=0.4):
    """
    Retrieves context but returns None if the context isn't relevant enough.
    Threshold: Lower is stricter (0.0 is exact match, 1.0 is everything).
    """
    try:
        query_vec = get_fireworks_embeddings([query])

        results = collection.query(
            query_embeddings=query_vec,
            n_results=3,
            include=["documents", "distances"]  # We need distances to check relevance
        )

        if results["documents"] and results["documents"][0]:
            # NOTE: We are currently not filtering by distance threshold here.
            # You can optionally check results["distances"][0][0] against `threshold`
            # if you want stricter behavior.

            # Combine the documents
            context_text = "\n---\n".join(results["documents"][0])
            return context_text

        return None
    except Exception as e:
        print(f"Retrieval Error: {e}")
        return None

# ---- Simple Short-Term Memory ----
conversation_history = []  # list of {"role": "user"/"assistant", "content": str}

def add_message(role: str, content: str):
    """Save a message to the conversation history."""
    conversation_history.append({"role": role, "content": content})

def get_recent_history(max_turns: int = 6) -> str:
    """
    Return the last N messages (user + assistant) as a formatted text block.
    This will be included in the prompt as chat history.
    """
    recent_msgs = conversation_history[-max_turns:]
    lines = []
    for msg in recent_msgs:
        speaker = "User" if msg["role"] == "user" else "Assistant"
        lines.append(f"{speaker}: {msg['content']}")
    return "\n".join(lines)

def reset_history():
    """Call this when starting a new session / user (optional)."""
    conversation_history.clear()
# ----------------------------------

# 3. Chat Loop
print("💬 --- Strict Chat System Ready (Type 'exit' to stop) ---")

# Strict System Instruction
system_instruction = """
You are a strict AI assistant. You must answer the user's question strictly based ONLY on the provided Context.
Rules:
0. Greetings & General Help: If the user says "Hello", "Can you help?", "Thanks", or generic phrases, reply politely and professionally in Arabic. You do NOT need context for this.
1. Do not use any outside knowledge or pre-trained information.
2. If the answer is found in the Context, answer in Arabic.
3. If the answer is NOT in the Context, you must strictly say: "عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال."
4. Do not make up facts.
"""

while True:
    user_input = input("\n👤 You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # 1) احفظ رسالة المستخدم في التاريخ
    add_message("user", user_input)

    # 2) استرجع السياق من الـ RAG باستخدام السؤال الحالي فقط
    context = retrieve_strict_context(user_input)

    # إذا ما فيه سياق مناسب، نطلع برد ثابت
    if not context:
        no_context_reply = "عذراً، لم أجد أي معلومات مطابقة في قاعدة البيانات."
        print(f"🤖 AI: {no_context_reply}")
        # نحفظ الرد في الهستوري برضه لو حابين
        add_message("assistant", no_context_reply)
        continue

    # 3) جهّز الـ history النصي (آخر N رسائل فقط)
    history_text = get_recent_history(max_turns=6)

    # 4) نبني الـ prompt: history + context + السؤال الحالي
    prompt = f"""
Conversation history (last turns):
{history_text}

Context (retrieved from the knowledge base):
{context}

Current user question:
{user_input}
"""

    try:
        response = client.chat.completions.create(
            model="accounts/fireworks/models/qwen2p5-vl-32b-instruct",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": prompt},
            ],
            # Temperature 0.0 = no creativity (good for strict RAG)
            temperature=0.0,
            max_tokens=200,
        )

        answer = response.choices[0].message.content
        print(f"🤖 AI: {answer}")

        # 5) احفظ رد المساعد في التاريخ
        add_message("assistant", answer)

    except Exception as e:
        print(f"❌ Error: {e}")'''

'# Cell 5: Strict Chat System (Anti-Hallucination)\nimport chromadb\n\n# 1. Connect to Database\ndb_client = chromadb.PersistentClient(path="./my_arabic_db")\ncollection = db_client.get_collection(name="grammar_collection")\n\n# 2. Retrieval Function with Similarity Check (same as before)\ndef retrieve_strict_context(query, threshold=0.4):\n    """\n    Retrieves context but returns None if the context isn\'t relevant enough.\n    Threshold: Lower is stricter (0.0 is exact match, 1.0 is everything).\n    """\n    try:\n        query_vec = get_fireworks_embeddings([query])\n\n        results = collection.query(\n            query_embeddings=query_vec,\n            n_results=3,\n            include=["documents", "distances"]  # We need distances to check relevance\n        )\n\n        if results["documents"] and results["documents"][0]:\n            # NOTE: We are currently not filtering by distance threshold here.\n            # You can optionally check results["distances"][0][0] again

In [8]:
# Cell 5: Strict Chat System (Anti-Hallucination + Short-Term Memory)
import chromadb

# 1. Connect to Database
db_client = chromadb.PersistentClient(path="./my_arabic_db")
collection = db_client.get_collection(name="grammar_collection")

# 2. Retrieval Function with Similarity Check
def retrieve_strict_context(query, threshold=0.4):
    """
    Retrieves context but returns None if the context isn't relevant enough.
    Threshold: Lower is stricter (0.0 is exact match, 1.0 is everything).
    """
    try:
        query_vec = get_fireworks_embeddings([query])

        results = collection.query(
            query_embeddings=query_vec,
            n_results=3,
            include=["documents", "distances"]  # We need distances to check relevance
        )

        if results["documents"] and results["documents"][0]:
            # NOTE: We are currently not filtering by distance threshold here.
            # You can optionally check results["distances"][0][0] against `threshold`
            # if you want stricter behavior.

            # Combine the documents
            context_text = "\n---\n".join(results["documents"][0])
            return context_text

        return None
    except Exception as e:
        print(f"Retrieval Error: {e}")
        return None

# ---- Simple Short-Term Memory ----
conversation_history = []  # list of {"role": "user"/"assistant", "content": str}

def add_message(role: str, content: str):
    """Save a message to the conversation history."""
    conversation_history.append({"role": role, "content": content})

def get_recent_history(max_turns: int = 6) -> str:
    """
    Return the last N messages (user + assistant) as a formatted text block.
    This will be included in the prompt as chat history.
    """
    recent_msgs = conversation_history[-max_turns:]
    lines = []
    for msg in recent_msgs:
        speaker = "User" if msg["role"] == "user" else "Assistant"
        lines.append(f"{speaker}: {msg['content']}")
    return "\n".join(lines)

def reset_history():
    """Call this when starting a new session / user (optional)."""
    conversation_history.clear()
# ----------------------------------

# 3. System Instruction
system_instruction = """
You are a strict AI assistant. You must answer the user's question strictly based ONLY on the provided Context.
Rules:
0. Greetings & General Help: If the user says "Hello", "Can you help?", "Thanks", or generic phrases, reply politely and professionally in Arabic. You do NOT need context for this.
1. Do not use any outside knowledge or pre-trained information.
2. If the answer is found in the Context, answer in Arabic.
3. If the answer is NOT in the Context, you must strictly say: "عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال."
4. Do not make up facts.
"""

print("💬 Strict Chat System with Memory is loaded. Use chat_with_memory('سؤالك') to talk to the model.")

def chat_with_memory(user_input: str):
    """Send one turn to the RAG chatbot with short-term memory."""
    # 1) احفظ رسالة المستخدم في التاريخ
    add_message("user", user_input)

    # 2) استرجع السياق من الـ RAG باستخدام السؤال الحالي فقط
    context = retrieve_strict_context(user_input)

    # إذا ما فيه سياق مناسب، نطلع برد ثابت
    if not context:
        no_context_reply = "عذراً، لم أجد أي معلومات مطابقة في قاعدة البيانات."
        print(f"🤖 AI: {no_context_reply}")
        add_message("assistant", no_context_reply)
        return no_context_reply

    # 3) جهّز الـ history النصي (آخر N رسائل فقط)
    history_text = get_recent_history(max_turns=6)

    # 4) نبني الـ prompt: history + context + السؤال الحالي
    prompt = f"""
Conversation history (last turns):
{history_text}

Context (retrieved from the knowledge base):
{context}

Current user question:
{user_input}
"""

    try:
        response = client.chat.completions.create(
            model="accounts/fireworks/models/qwen2p5-vl-32b-instruct",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": prompt},
            ],
            temperature=0.0,  # no creativity (strict RAG)
            max_tokens=200,
        )

        answer = response.choices[0].message.content
        print(f"🤖 AI: {answer}")

        # 5) احفظ رد المساعد في التاريخ
        add_message("assistant", answer)
        return answer

    except Exception as e:
        print(f"❌ Error: {e}")
        return None

💬 Strict Chat System with Memory is loaded. Use chat_with_memory('سؤالك') to talk to the model.


In [9]:
chat_with_memory('اذكر قاعده من القواعد المنهجيه العامه في القياس عند النحويين؟')

🤖 AI: الاصل عدم مخالفه القياس.


'الاصل عدم مخالفه القياس.'

In [10]:
chat_with_memory("وضح أكثر المثال السابق")

🤖 AI: عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.


'عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.'

In [11]:
#عشان اشوف اذا فعلًا متسلسل ولا لا
print(get_recent_history())

User: اذكر قاعده من القواعد المنهجيه العامه في القياس عند النحويين؟
Assistant: الاصل عدم مخالفه القياس.
User: وضح أكثر المثال السابق
Assistant: عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.


In [12]:
chat_with_memory("هل توجد قاعدة مختلفة عن ( الأصل عدم مخالفة القياس؟)")

🤖 AI: نعم، توجد قاعدة مختلفة عن "الأصل عدم مخالفة القياس". من القواعد المنهجية العامة في القياس عند النحويين: "ليس شرطا في القياس مساواة المقيس والمقيس عليه في جميع الأحكام."


'نعم، توجد قاعدة مختلفة عن "الأصل عدم مخالفة القياس". من القواعد المنهجية العامة في القياس عند النحويين: "ليس شرطا في القياس مساواة المقيس والمقيس عليه في جميع الأحكام."'

In [13]:
#تصفير للميموري
reset_history()

In [14]:
chat_with_memory('ما الاصل في كلمه (الكتاب)؟')

🤖 AI: (الكتاب) في الاصل مصدر (كتب)، ثم صار اسما للقران الكريم.


'(الكتاب) في الاصل مصدر (كتب)، ثم صار اسما للقران الكريم.'

In [15]:
chat_with_memory("مالذي اصبح اسم للقران الكريم؟")

🤖 AI: الكتاب.


'الكتاب.'

In [16]:
#عشان اشوف اذا فعلًا متسلسل ولا لا
print(get_recent_history())

User: ما الاصل في كلمه (الكتاب)؟
Assistant: (الكتاب) في الاصل مصدر (كتب)، ثم صار اسما للقران الكريم.
User: مالذي اصبح اسم للقران الكريم؟
Assistant: الكتاب.


In [17]:
chat_with_memory("ما الفرق بين المعرب المنصرف والمعرب غير المنصرف.")

🤖 AI: المعرب المنصرف هو الاسم الذي يمكن أن يتغير أعرابه ويقبل التنوين، مثل "علي". أما المعرب غير المنصرف فهو الاسم الذي تتغير حالته الاعرابية ولكنه لا يقبل التنوين، مثل "أحمد" و"فاطمة".


'المعرب المنصرف هو الاسم الذي يمكن أن يتغير أعرابه ويقبل التنوين، مثل "علي". أما المعرب غير المنصرف فهو الاسم الذي تتغير حالته الاعرابية ولكنه لا يقبل التنوين، مثل "أحمد" و"فاطمة".'

In [18]:
chat_with_memory("اذكر مثال")

🤖 AI: الإجابة:  
الإجابة تعتمد على السؤال الذي يُطلب فيه المثال. بناءً على السياق المتوفر، يمكن أن يكون المثال متعلقًا بالعرض أو الإضافة أو الإضافة "اذ" إلى الجملة الفعلية. إذا كنت تقصد مثالًا معينًا، يرجى توضيح السؤال.  

إذا كنت تقصد مثالًا عامًا، يمكن أن يكون:  
"الإجابة تعتمد على السؤال الذي يُطلب فيه المثال."  

إذا كنت تقصد مثالًا محددًا من السياق، يرجى توضيح السؤال.  

عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.


'الإجابة:  \nالإجابة تعتمد على السؤال الذي يُطلب فيه المثال. بناءً على السياق المتوفر، يمكن أن يكون المثال متعلقًا بالعرض أو الإضافة أو الإضافة "اذ" إلى الجملة الفعلية. إذا كنت تقصد مثالًا معينًا، يرجى توضيح السؤال.  \n\nإذا كنت تقصد مثالًا عامًا، يمكن أن يكون:  \n"الإجابة تعتمد على السؤال الذي يُطلب فيه المثال."  \n\nإذا كنت تقصد مثالًا محددًا من السياق، يرجى توضيح السؤال.  \n\nعذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.'

In [19]:
chat_with_memory("احمد و فاطمه ، مثال للسؤال السابق؟")

🤖 AI: عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.


'عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.'

In [20]:
#عشان اشوف اذا فعلًا متسلسل ولا لا
print(get_recent_history())

User: ما الفرق بين المعرب المنصرف والمعرب غير المنصرف.
Assistant: المعرب المنصرف هو الاسم الذي يمكن أن يتغير أعرابه ويقبل التنوين، مثل "علي". أما المعرب غير المنصرف فهو الاسم الذي تتغير حالته الاعرابية ولكنه لا يقبل التنوين، مثل "أحمد" و"فاطمة".
User: اذكر مثال
Assistant: الإجابة:  
الإجابة تعتمد على السؤال الذي يُطلب فيه المثال. بناءً على السياق المتوفر، يمكن أن يكون المثال متعلقًا بالعرض أو الإضافة أو الإضافة "اذ" إلى الجملة الفعلية. إذا كنت تقصد مثالًا معينًا، يرجى توضيح السؤال.  

إذا كنت تقصد مثالًا عامًا، يمكن أن يكون:  
"الإجابة تعتمد على السؤال الذي يُطلب فيه المثال."  

إذا كنت تقصد مثالًا محددًا من السياق، يرجى توضيح السؤال.  

عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.
User: احمد و فاطمه ، مثال للسؤال السابق؟
Assistant: عذراً، لا توجد معلومات كافية في البيانات للإجابة على هذا السؤال.


In [21]:
#تصفير للميموري
reset_history()

بعد الرن لـ CEL 5

New Cell:

chat_with_memory("السؤال هنا")

عشان نشوف إذا متسلسل فعليًا أو لا؟ نسوي Print

أخيرًا نسوي Reset

reset_history()




NOTE

*الميموري قصير المدى (آخر ٦ رسائل تقريبًا)، وما فيه تخزين دائم في قاعدة بيانات.


In [22]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os
import json
import random
import time
import sys
import re
import pandas as pd
from groq import Groq
import pandas as pd
import matplotlib.pyplot as plt

# ================== CONFIG ==================
API_KEY = "API Key"  # better: use env var
MODEL_NAME = "llama-3.1-8b-instant"
DATASET_PATH = "rag_dataset_final.json"
SAMPLE_PERCENT = 0.01  # 1% sample; increase carefully
OUTPUT_FILE = "evaluation_results.csv"

groq_client = Groq(api_key=API_KEY)

# ================== LOAD DATA ==================
with open(DATASET_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

sample_size = max(1, int(len(data) * SAMPLE_PERCENT))
dataset = random.sample(data, sample_size)
total = len(dataset)

print(f"📌 Evaluating {total} samples\n")

# ================== HELPERS ==================
def extract_json(text: str) -> str:
    """Try to extract the first {...} JSON object from model output."""
    match = re.search(r"\{.*\}", text, re.DOTALL)
    return match.group(0) if match else text

def simple_overlap(a: str, b: str) -> float:
    """Naive token overlap between two strings (works fine for Arabic too)."""
    tokenize = lambda s: set(re.findall(r"\w+", s.lower()))
    ta, tb = tokenize(a), tokenize(b)
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)


# ================== EVAL PROMPT ==================
EVAL_PROMPT = """
You are evaluating a Retrieval-Augmented Generation (RAG) system.

You are given:
- A user question
- Retrieved context from a knowledge base
- The model's answer
- The reference (ground truth) answer

Score each metric from 0 to 3:

- context_relevance: Are the retrieved contexts relevant to the question?
- context_precision: How much of the retrieved text is actually useful?
- answer_relevance: Does the model answer the user's question?
- faithfulness: Is the model answer supported by the retrieved context only?
- correctness: How close is the model answer to the reference answer?

Return ONLY valid JSON with this exact structure:

{{
  "context_relevance": number,
  "context_precision": number,
  "answer_relevance": number,
  "faithfulness": number,
  "correctness": number,
  "comments": "short comment in Arabic about main issues"
}}

Question:
{question}

Reference Answer:
{reference}

Retrieved Context:
{context}

Model Answer:
{answer}
"""




📌 Evaluating 83 samples



In [24]:


# ================== GROQ JUDGE ==================
def evaluate_with_groq(question: str, reference: str, context, answer: str) -> dict:
    """Ask Groq model to score a single QA+context triple."""
    ctx_text = context if isinstance(context, str) else "\n\n".join(context)

    prompt = EVAL_PROMPT.format(
        question=question,
        reference=reference,
        context=ctx_text,
        answer=answer,
    )

    completion = groq_client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {
                "role": "system",
                "content": "You are an accurate RAG evaluation assistant. You MUST respond only with JSON, no extra text.",
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )

    response_text = completion.choices[0].message.content
    response_text = extract_json(response_text)

    try:
        data = json.loads(response_text)
        # ensure all keys exist
        for k in [
            "context_relevance",
            "context_precision",
            "answer_relevance",
            "faithfulness",
            "correctness",
            "comments",
        ]:
            if k not in data:
                if k == "comments":
                    data[k] = ""
                else:
                    data[k] = 0
        return data
    except json.JSONDecodeError:
        return {
            "context_relevance": 0,
            "context_precision": 0,
            "answer_relevance": 0,
            "faithfulness": 0,
            "correctness": 0,
            "comments": f"⚠ Could not parse judge output: {response_text}",
        }



In [25]:
# ================== SAFE WRAPPER WITH RATE-LIMIT HANDLING ==================
rate_limit_count = 0

def safe_rag_answer(q: str, max_retries: int = 5, base_delay: float = 1.0):
    """
    Wrap rag_answer(q) with retry logic on Fireworks/Groq rate limits (429).
    Fast normally; slows down only when rate limit happens.
    """
    global rate_limit_count

    for attempt in range(max_retries):
        try:
            x = chat_with_memory(q)
            reset_history()
            return x

        except Exception as e:
            msg = str(e).lower()

            is_rate_limit = any(
                key in msg
                for key in [
                    "429",
                    "rate limit",
                    "rate_limit",
                    "rate_limit_exceeded",
                    "request rate limit exceeded",
                ]
            )

            if not is_rate_limit:
                # Non-rate-limit error -> let outer loop mark as skipped
                print(f"\n❌ Non rate-limit error in rag_answer:")
                print("   Error type:", type(e).__name__)
                print("   Error msg :", e)
                raise

            rate_limit_count += 1
            delay = base_delay * (2 ** attempt)  # 1s, 2s, 4s, ...
            print(f"\n⚠ Rate limit detected, backing off for {delay:.1f}s...")

    raise RuntimeError("Too many rate-limit failures for this question.")



In [ ]:
import os
import json
import random
import time
import sys
import re
import pandas as pd
from groq import Groq

# ================== CONFIG ==================
API_KEY = "API Key"
MODEL_NAME = "llama-3.1-8b-instant"
DATASET_PATH = "rag_dataset_final.json"
SAMPLE_PERCENT = 0.05  # 1% sample
OUTPUT_FILE = "results/eval_results.csv"

groq_client = Groq(api_key=API_KEY)

# ================== LOAD DATA ==================
with open(DATASET_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

sample_size = max(1, int(len(data) * SAMPLE_PERCENT))
dataset = random.sample(data, sample_size)
total = len(dataset)

print(f"📌 Evaluating {total} samples\n")

# ================== HELPERS ==================
def extract_json(text: str) -> str:
    """Try to extract the first {...} JSON object from model output."""
    match = re.search(r"\{.*\}", text, re.DOTALL)
    return match.group(0) if match else text

def simple_overlap(a: str, b: str) -> float:
    """Naive token overlap between two strings."""
    tokenize = lambda s: set(re.findall(r"\w+", s.lower()))
    ta, tb = tokenize(a), tokenize(b)
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)

# ================== EVAL PROMPT ==================
EVAL_PROMPT_WITH_EXAMPLES = """
You are an expert evaluator for Retrieval-Augmented Generation (RAG) systems.

=== EVALUATION METRICS (Scale 0-3) ===

1. **Context Relevance**: Do contexts relate to the question?
   - 3: Perfect match to question topic
   - 2: Related with some relevant info
   - 1: Tangentially related
   - 0: Unrelated

2. **Context Precision**: Useful info vs noise ratio?
   - 3: >80% useful content
   - 2: 50-80% useful content
   - 1: 20-50% useful content
   - 0: <20% useful content

3. **Answer Relevance**: Does answer address question?
   - 3: Directly and completely
   - 2: Addresses but incomplete
   - 1: Partially addresses
   - 0: Doesn't address

4. **Faithfulness**: Answer based on context only?
   - 3: 100% from context
   - 2: Mostly from context (>80%)
   - 1: Some from context (50-80%)
   - 0: Mostly hallucinated (<50%)

5. **Correctness**: Match with reference answer?
   - 3: Semantically equivalent
   - 2: Core meaning correct
   - 1: Partially correct
   - 0: Wrong or opposite

=== EXAMPLE ===

Question: "ما هي الجملة الاسمية؟"
Context: "الجملة الاسمية تبدأ باسم وتتكون من مبتدأ وخبر"
Model: "الجملة الاسمية تبدأ باسم"
Reference: "الجملة الاسمية هي التي تبدأ باسم"

Scores:
- context_relevance: 3 (context perfectly matches question)
- context_precision: 3 (all information is useful)
- answer_relevance: 3 (directly answers the question)
- faithfulness: 3 (answer is directly from context)
- correctness: 2 (correct but less detailed than reference)

=== YOUR TASK ===

Question:
{question}

Reference Answer:
{reference}

Retrieved Context:
{context}

Model Answer:
{answer}

=== OUTPUT ===

Return ONLY this JSON (no markdown, no extra text):

{{
  "context_relevance": <0-3>,
  "context_precision": <0-3>,
  "answer_relevance": <0-3>,
  "faithfulness": <0-3>,
  "correctness": <0-3>,
  "comments": "<Arabic comment>"
}}
"""

# ================== GROQ JUDGE ==================
def evaluate_with_groq(question: str, reference: str, context, answer: str) -> dict:
    """Ask Groq model to score a single QA+context triple."""
    ctx_text = context if isinstance(context, str) else "\n\n".join(context)

    prompt = EVAL_PROMPT.format(
        question=question,
        reference=reference,
        context=ctx_text,
        answer=answer,
    )

    completion = groq_client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {
                "role": "system",
                "content": "You are an accurate RAG evaluation assistant. You MUST respond only with JSON, no extra text.",
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )

    response_text = completion.choices[0].message.content
    response_text = extract_json(response_text)

    try:
        data = json.loads(response_text)
        # ensure all keys exist
        for k in [
            "context_relevance",
            "context_precision",
            "answer_relevance",
            "faithfulness",
            "correctness",
            "comments",
        ]:
            if k not in data:
                if k == "comments":
                    data[k] = ""
                else:
                    data[k] = 0
        return data
    except json.JSONDecodeError:
        return {
            "context_relevance": 0,
            "context_precision": 0,
            "answer_relevance": 0,
            "faithfulness": 0,
            "correctness": 0,
            "comments": f"⚠ Could not parse judge output: {response_text}",
        }

# ================== MODIFIED RAG ANSWER WITH CONTEXT RETURN ==================
def retrieve_strict_context(query, threshold=0.4):
    """
    Retrieves context and returns both the context text and raw results.
    """
    try:
        query_vec = get_fireworks_embeddings([query])

        results = collection.query(
            query_embeddings=query_vec,
            n_results=3,
            include=["documents", "distances"]
        )

        if results["documents"] and results["documents"][0]:
            context_text = "\n---\n".join(results["documents"][0])
            return context_text, results["documents"][0]  # Return both combined and list

        return None, []
    except Exception as e:
        print(f"Retrieval Error: {e}")
        return None, []

def chat_with_memory_and_context(user_input: str):
    """
    Modified version that returns BOTH answer AND retrieved contexts.
    """
    # 1) Save user message
    add_message("user", user_input)

    # 2) Retrieve context
    context, context_list = retrieve_strict_context(user_input)

    # If no context found
    if not context:
        no_context_reply = "عذراً، لم أجد أي معلومات مطابقة في قاعدة البيانات."
        add_message("assistant", no_context_reply)
        return no_context_reply, []

    # 3) Get conversation history
    history_text = get_recent_history(max_turns=6)

    # 4) Build prompt
    prompt = f"""
Conversation history (last turns):
{history_text}

Context (retrieved from the knowledge base):
{context}

Current user question:
{user_input}
"""

    try:
        response = client.chat.completions.create(
            model="accounts/fireworks/models/qwen2p5-vl-32b-instruct",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": prompt},
            ],
            temperature=0.0,
            max_tokens=200,
        )

        answer = response.choices[0].message.content

        # 5) Save assistant response
        add_message("assistant", answer)

        return answer, context_list  # Return both answer and contexts

    except Exception as e:
        print(f"❌ Error: {e}")
        raise  # Re-raise to handle in safe_rag_answer

# ================== SAFE WRAPPER WITH RATE-LIMIT HANDLING ==================
rate_limit_count = 0

def safe_rag_answer(q: str, max_retries: int = 5, base_delay: float = 1.0):
    """
    Wrap RAG answer with retry logic on rate limits.
    Returns (answer, contexts) tuple.
    """
    global rate_limit_count

    for attempt in range(max_retries):
        try:
            answer, contexts = chat_with_memory_and_context(q)
            reset_history()
            return answer, contexts

        except Exception as e:
            msg = str(e).lower()

            is_rate_limit = any(
                key in msg
                for key in [
                    "429",
                    "rate limit",
                    "rate_limit",
                    "rate_limit_exceeded",
                    "request rate limit exceeded",
                ]
            )

            if not is_rate_limit:
                # Non-rate-limit error
                print(f"\n❌ Non rate-limit error in rag_answer:")
                print("   Error type:", type(e).__name__)
                print("   Error msg:", e)
                raise

            rate_limit_count += 1
            delay = base_delay * (2 ** attempt)
            print(f"\n⚠ Rate limit detected, backing off for {delay:.1f}s...")
            time.sleep(delay)

    raise RuntimeError("Too many rate-limit failures for this question.")

# ================== MAIN EVAL LOOP ==================
os.makedirs("results", exist_ok=True)

results = []
success_count = 0
skipped_count = 0
processed_ids = set()

# Load previous results if exists
if os.path.exists(OUTPUT_FILE):
    prev_df = pd.read_csv(OUTPUT_FILE)
    if "id" in prev_df.columns:
        processed_ids = set(prev_df["id"].tolist())
    results = prev_df.to_dict(orient="records")
    success_count = len(results)
    print(f"🔁 Resuming: found {success_count} existing results in {OUTPUT_FILE}")
else:
    print("🆕 Starting fresh, no previous results found.")

def status_line():
    sys.stdout.write(
        f"\rProgress: {success_count}/{total} | "
        f"Skipped: {skipped_count} | "
        f"Rate-Limits: {rate_limit_count}"
    )
    sys.stdout.flush()

status_line()

for i, row in enumerate(dataset):
    # Skip already processed rows
    if i in processed_ids:
        continue

    question = row["question_without_diacritics"]
    reference = row["answer_without_diacritics"]

    # 1) Retrieval + generation
    try:
        answer, contexts = safe_rag_answer(question)
    except Exception as e:
        print(f"\n⚠ Skipping sample {i} due to error: {e}")
        skipped_count += 1
        status_line()
        continue

    # 2) Model-based evaluation
    try:
        score = evaluate_with_groq(question, reference, contexts, answer)
    except Exception as e:
        print(f"\n⚠ Evaluation failed for sample {i}: {e}")
        skipped_count += 1
        status_line()
        continue

    # 3) Lexical heuristic
    overlap = simple_overlap(answer, reference)

    # Normalize contexts for logging
    if isinstance(contexts, str):
        ctx_joined = contexts
    else:
        ctx_joined = " | ".join(contexts) if contexts else "No context"

    # 4) Collect row
    row_result = {
        "id": i,
        "question": question,
        "reference": reference,
        "model_answer": answer,
        "retrieved_contexts": ctx_joined,
        "context_relevance": score["context_relevance"],
        "context_precision": score["context_precision"],
        "answer_relevance": score["answer_relevance"],
        "faithfulness": score["faithfulness"],
        "correctness": score["correctness"],
        "lexical_overlap": round(overlap, 3),
        "comments": score["comments"],
        "final_score": round(
            (
                score["context_relevance"]
                + score["context_precision"]
                + score["answer_relevance"]
                + score["faithfulness"]
                + score["correctness"]
            )
            / 5,
            3,
        ),
    }

    results.append(row_result)
    success_count += 1

    # 5) Save after each successful row
    df = pd.DataFrame(results)
    df.to_csv(OUTPUT_FILE, index=False)

    status_line()

# ================== FINAL SUMMARY ==================
print("\n\n✅ DONE")
print(f"📁 Saved → {OUTPUT_FILE}")
print(f"📊 Evaluated this run: {success_count}")
print(f"🚫 Skipped this run: {skipped_count}")
print(f"⚠ Rate-limit incidents: {rate_limit_count}")

📌 Evaluating 417 samples

🆕 Starting fresh, no previous results found.
Progress: 417/417 | Skipped: 0 | Rate-Limits: 0

✅ DONE
📁 Saved → results/eval_results.csv
📊 Evaluated this run: 417
🚫 Skipped this run: 0
⚠ Rate-limit incidents: 0


In [28]:
import pandas as pd
import numpy as np

# ==================== CONFIGURATION ====================
INPUT_FILE = "results/eval_results.csv"
OUTPUT_FILE = "results/eval_results_with_retrieval_metrics.csv"
K = 3  # Top K contexts to consider

# ==================== LOAD DATA ====================
print("Loading evaluation results...")
df = pd.read_csv(INPUT_FILE)
print(f"Loaded {len(df)} samples\n")

# ==================== RETRIEVAL METRICS FUNCTIONS ====================

def calculate_recall_at_k(retrieved_contexts, reference_answer, k=3):
    """Recall@K: What proportion of relevant information was retrieved?"""
    if not retrieved_contexts or not reference_answer:
        return 0.0

    ref_tokens = set(str(reference_answer).split())
    contexts_list = str(retrieved_contexts).split(' | ')[:k]

    context_tokens = set()
    for ctx in contexts_list:
        context_tokens.update(ctx.split())

    if len(ref_tokens) == 0:
        return 0.0

    relevant_found = len(ref_tokens & context_tokens)
    return relevant_found / len(ref_tokens)


def calculate_precision_at_k(retrieved_contexts, reference_answer, k=3):
    """Precision@K: What proportion of retrieved information is relevant?"""
    if not retrieved_contexts or not reference_answer:
        return 0.0

    ref_tokens = set(str(reference_answer).split())
    contexts_list = str(retrieved_contexts).split(' | ')[:k]

    context_tokens = set()
    for ctx in contexts_list:
        context_tokens.update(ctx.split())

    if len(context_tokens) == 0:
        return 0.0

    relevant_found = len(ref_tokens & context_tokens)
    return relevant_found / len(context_tokens)


def calculate_mrr(retrieved_contexts, reference_answer):
    """MRR: Mean Reciprocal Rank - Position of first relevant context"""
    if not retrieved_contexts or not reference_answer:
        return 0.0

    ref_tokens = set(str(reference_answer).split())
    contexts_list = str(retrieved_contexts).split(' | ')

    for rank, ctx in enumerate(contexts_list, start=1):
        ctx_tokens = set(ctx.split())
        overlap = len(ref_tokens & ctx_tokens)
        if overlap > len(ref_tokens) * 0.3:  # 30% overlap threshold
            return 1.0 / rank

    return 0.0


def calculate_ndcg_at_k(retrieved_contexts, reference_answer, k=3):
    """nDCG@K: Normalized Discounted Cumulative Gain"""
    if not retrieved_contexts or not reference_answer:
        return 0.0

    ref_tokens = set(str(reference_answer).split())
    contexts_list = str(retrieved_contexts).split(' | ')[:k]

    # Calculate relevance scores for each context (0-3 scale)
    relevance_scores = []
    for ctx in contexts_list:
        ctx_tokens = set(ctx.split())
        overlap = len(ref_tokens & ctx_tokens) / len(ref_tokens) if ref_tokens else 0

        if overlap > 0.7:
            relevance_scores.append(3)
        elif overlap > 0.4:
            relevance_scores.append(2)
        elif overlap > 0.2:
            relevance_scores.append(1)
        else:
            relevance_scores.append(0)

    if not relevance_scores:
        return 0.0

    # Calculate DCG
    dcg = relevance_scores[0]
    for i, score in enumerate(relevance_scores[1:], start=2):
        dcg += score / np.log2(i + 1)

    # Calculate ideal DCG
    ideal_scores = sorted(relevance_scores, reverse=True)
    idcg = ideal_scores[0]
    for i, score in enumerate(ideal_scores[1:], start=2):
        idcg += score / np.log2(i + 1)

    return dcg / idcg if idcg > 0 else 0.0

# ==================== CALCULATE METRICS ====================

print("Calculating retrieval metrics...")

df['recall_at_3'] = df.apply(
    lambda row: calculate_recall_at_k(row['retrieved_contexts'], row['reference'], k=K),
    axis=1
)

df['precision_at_3'] = df.apply(
    lambda row: calculate_precision_at_k(row['retrieved_contexts'], row['reference'], k=K),
    axis=1
)

df['mrr'] = df.apply(
    lambda row: calculate_mrr(row['retrieved_contexts'], row['reference']),
    axis=1
)

df['ndcg_at_3'] = df.apply(
    lambda row: calculate_ndcg_at_k(row['retrieved_contexts'], row['reference'], k=K),
    axis=1
)

print("Done!\n")

# ==================== SAVE RESULTS ====================

df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Saved results to: {OUTPUT_FILE}\n")

# ==================== DISPLAY SUMMARY ====================

print("=" * 60)
print("EVALUATION METRICS SUMMARY (Scale 0-1)")
print("=" * 60)

eval_metrics = ['context_relevance', 'context_precision', 'answer_relevance',
                'faithfulness', 'correctness', 'lexical_overlap']

print(f"\n{'Metric':<30} {'Mean':<15} {'Std':<15}")
print("-" * 60)

for metric in eval_metrics:
    if metric in df.columns:
        # Normalize to 0-1 scale (divide by 3 for metrics on 0-3 scale)
        if metric == 'lexical_overlap':
            # Already on 0-1 scale
            mean_val = df[metric].mean()
            std_val = df[metric].std()
        else:
            # Scale from 0-3 to 0-1
            mean_val = df[metric].mean() / 3.0
            std_val = df[metric].std() / 3.0

        print(f"{metric:<30} {mean_val:<15.3f} {std_val:<15.3f}")

print("\n" + "=" * 60)
print("RETRIEVAL METRICS SUMMARY (Scale 0-1)")
print("=" * 60)

retrieval_metrics = ['recall_at_3', 'precision_at_3', 'mrr', 'ndcg_at_3']

print(f"\n{'Metric':<30} {'Mean':<15} {'Std':<15}")
print("-" * 60)

for metric in retrieval_metrics:
    print(f"{metric:<30} "
          f"{df[metric].mean():<15.3f} "
          f"{df[metric].std():<15.3f}")

print("\n" + "=" * 60)
print("INTERPRETATION & INSIGHTS")
print("=" * 60)

# Evaluation Metrics Interpretation
print("\n📊 EVALUATION QUALITY METRICS:")
print("-" * 60)

if 'context_relevance' in df.columns:
    avg_ctx_rel = df['context_relevance'].mean() / 3.0  # Normalize to 0-1
    if avg_ctx_rel >= 0.83:
        print(f"✅ Context Relevance: {avg_ctx_rel:.3f} - Excellent")
    elif avg_ctx_rel >= 0.67:
        print(f"✅ Context Relevance: {avg_ctx_rel:.3f} - Good")
    elif avg_ctx_rel >= 0.50:
        print(f"🟡 Context Relevance: {avg_ctx_rel:.3f} - Fair")
    else:
        print(f"⚠️  Context Relevance: {avg_ctx_rel:.3f} - Poor")

if 'context_precision' in df.columns:
    avg_ctx_prec = df['context_precision'].mean() / 3.0
    if avg_ctx_prec >= 0.83:
        print(f"✅ Context Precision: {avg_ctx_prec:.3f} - Excellent")
    elif avg_ctx_prec >= 0.67:
        print(f"✅ Context Precision: {avg_ctx_prec:.3f} - Good")
    elif avg_ctx_prec >= 0.50:
        print(f"🟡 Context Precision: {avg_ctx_prec:.3f} - Fair")
    else:
        print(f"⚠️  Context Precision: {avg_ctx_prec:.3f} - Poor")

if 'answer_relevance' in df.columns:
    avg_ans_rel = df['answer_relevance'].mean() / 3.0
    if avg_ans_rel >= 0.83:
        print(f"✅ Answer Relevance: {avg_ans_rel:.3f} - Excellent")
    elif avg_ans_rel >= 0.67:
        print(f"✅ Answer Relevance: {avg_ans_rel:.3f} - Good")
    elif avg_ans_rel >= 0.50:
        print(f"🟡 Answer Relevance: {avg_ans_rel:.3f} - Fair")
    else:
        print(f"⚠️  Answer Relevance: {avg_ans_rel:.3f} - Poor")

if 'faithfulness' in df.columns:
    avg_faith = df['faithfulness'].mean() / 3.0
    if avg_faith >= 0.83:
        print(f"✅ Faithfulness: {avg_faith:.3f} - Excellent (minimal hallucination)")
    elif avg_faith >= 0.67:
        print(f"✅ Faithfulness: {avg_faith:.3f} - Good")
    elif avg_faith >= 0.50:
        print(f"🟡 Faithfulness: {avg_faith:.3f} - Fair (some hallucination)")
    else:
        print(f"⚠️  Faithfulness: {avg_faith:.3f} - Poor (significant hallucination)")

if 'correctness' in df.columns:
    avg_correct = df['correctness'].mean() / 3.0
    if avg_correct >= 0.83:
        print(f"✅ Correctness: {avg_correct:.3f} - Excellent")
    elif avg_correct >= 0.67:
        print(f"✅ Correctness: {avg_correct:.3f} - Good")
    elif avg_correct >= 0.50:
        print(f"🟡 Correctness: {avg_correct:.3f} - Fair")
    else:
        print(f"⚠️  Correctness: {avg_correct:.3f} - Poor")

if 'lexical_overlap' in df.columns:
    avg_lex = df['lexical_overlap'].mean()
    if avg_lex >= 0.7:
        print(f"✅ Lexical Overlap: {avg_lex:.3f} - Excellent (high word overlap)")
    elif avg_lex >= 0.5:
        print(f"✅ Lexical Overlap: {avg_lex:.3f} - Good")
    elif avg_lex >= 0.3:
        print(f"🟡 Lexical Overlap: {avg_lex:.3f} - Fair")
    else:
        print(f"⚠️  Lexical Overlap: {avg_lex:.3f} - Poor (very different wording)")

# Retrieval Metrics Interpretation
print("\n🎯 RETRIEVAL PERFORMANCE METRICS:")
print("-" * 60)

avg_recall = df['recall_at_3'].mean()
avg_precision = df['precision_at_3'].mean()
avg_mrr = df['mrr'].mean()
avg_ndcg = df['ndcg_at_3'].mean()

if avg_recall < 0.3:
    print(f"⚠️  Recall@3: {avg_recall:.3f} - Poor (missing most relevant information)")
elif avg_recall < 0.5:
    print(f"🟡 Recall@3: {avg_recall:.3f} - Fair (missing some relevant information)")
else:
    print(f"✅ Recall@3: {avg_recall:.3f} - Good (capturing most relevant information)")

if avg_precision < 0.3:
    print(f"⚠️  Precision@3: {avg_precision:.3f} - Poor (too much noise in retrieval)")
elif avg_precision < 0.5:
    print(f"🟡 Precision@3: {avg_precision:.3f} - Fair (moderate noise)")
else:
    print(f"✅ Precision@3: {avg_precision:.3f} - Good (minimal noise)")

if avg_mrr < 0.4:
    print(f"⚠️  MRR: {avg_mrr:.3f} - Poor (relevant docs not ranked well)")
elif avg_mrr < 0.6:
    print(f"🟡 MRR: {avg_mrr:.3f} - Fair (moderate ranking quality)")
else:
    print(f"✅ MRR: {avg_mrr:.3f} - Good (good ranking quality)")

if avg_ndcg < 0.4:
    print(f"⚠️  nDCG@3: {avg_ndcg:.3f} - Poor (poor overall ranking)")
elif avg_ndcg < 0.6:
    print(f"🟡 nDCG@3: {avg_ndcg:.3f} - Fair (moderate overall ranking)")
else:
    print(f"✅ nDCG@3: {avg_ndcg:.3f} - Good (good overall ranking)")

# Overall System Analysis
print("\n💡 OVERALL SYSTEM ANALYSIS:")
print("-" * 60)

# Calculate overall score if final_score exists
if 'final_score' in df.columns:
    overall_score = df['final_score'].mean() / 3.0  # Normalize to 0-1
    print(f"\nOverall Quality Score: {overall_score:.3f}")

# Identify main bottleneck
bottlenecks = []
strengths = []

if 'context_relevance' in df.columns and df['context_relevance'].mean() / 3.0 < 0.67:
    bottlenecks.append("Context Relevance")
if 'answer_relevance' in df.columns and df['answer_relevance'].mean() / 3.0 >= 0.83:
    strengths.append("Answer Relevance")
if 'faithfulness' in df.columns and df['faithfulness'].mean() / 3.0 >= 0.83:
    strengths.append("Faithfulness")
if avg_recall < 0.4:
    bottlenecks.append("Retrieval Recall")
if avg_precision < 0.4:
    bottlenecks.append("Retrieval Precision")

if strengths:
    print(f"\n✅ Strengths: {', '.join(strengths)}")
if bottlenecks:
    print(f"⚠️  Bottlenecks: {', '.join(bottlenecks)}")

# Actionable recommendations
print("\n📋 RECOMMENDATIONS:")
print("-" * 60)

if avg_recall < 0.5:
    print("1. Improve Recall: Increase number of retrieved documents (n_results)")
if avg_precision < 0.5:
    print("2. Improve Precision: Add relevance filtering or reduce chunk size")
if avg_mrr < 0.5:
    print("3. Improve Ranking: Implement reranking (e.g., CrossEncoder)")
if 'correctness' in df.columns and df['correctness'].mean() / 3.0 < 0.67:
    print("4. Improve Correctness: Better align model outputs with references")
if 'faithfulness' in df.columns and df['faithfulness'].mean() / 3.0 < 0.83:
    print("5. Reduce Hallucination: Strengthen context grounding in prompts")

print("\n" + "=" * 60)

# ==================== VISUALIZATIONS ====================

import matplotlib.pyplot as plt

print("\nGenerating visualizations...")

# Create figure with subplots
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
fig.suptitle('RAG Evaluation Metrics Distribution', fontsize=16, fontweight='bold')

# Flatten axes for easier iteration
axes = axes.flatten()

# All metrics to plot
all_metrics = eval_metrics + retrieval_metrics

# Plot each metric
for idx, metric in enumerate(all_metrics):
    if metric in df.columns:
        ax = axes[idx]

        # Normalize evaluation metrics to 0-1 for consistency
        if metric in eval_metrics and metric != 'lexical_overlap':
            data = df[metric] / 3.0
            scale_label = "(0-1)"
        else:
            data = df[metric]
            scale_label = "(0-1)"

        # Create histogram
        ax.hist(data, bins=20, color='steelblue', edgecolor='black', alpha=0.7)

        # Add mean line
        mean_val = data.mean()
        ax.axvline(mean_val, color='red', linestyle='--', linewidth=2,
                   label=f'Mean: {mean_val:.3f}')

        # Formatting
        ax.set_title(metric.replace('_', ' ').title(), fontweight='bold', fontsize=10)
        ax.set_xlabel(f'Score {scale_label}', fontsize=9)
        ax.set_ylabel('Frequency', fontsize=9)
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)

# Remove empty subplots if any
for idx in range(len(all_metrics), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('results/metrics_distributions.png', dpi=300, bbox_inches='tight')
print("✅ Saved visualizations to: results/metrics_distributions.png")

plt.close()

print("\n" + "=" * 60)
print("COMPLETE! All metrics calculated and visualized.")
print("=" * 60)

Loading evaluation results...
Loaded 417 samples

Calculating retrieval metrics...
Done!

✅ Saved results to: results/eval_results_with_retrieval_metrics.csv

EVALUATION METRICS SUMMARY (Scale 0-1)

Metric                         Mean            Std            
------------------------------------------------------------
context_relevance              0.990           0.090          
context_precision              0.787           0.181          
answer_relevance               0.980           0.132          
faithfulness                   0.925           0.220          
correctness                    0.923           0.224          
lexical_overlap                0.723           0.327          

RETRIEVAL METRICS SUMMARY (Scale 0-1)

Metric                         Mean            Std            
------------------------------------------------------------
recall_at_3                    0.943           0.203          
precision_at_3                 0.279           0.132          
mrr      